In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import sklearn
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV

import tensorflow as tf


np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
!git clone https://github.com/MLinApp-FP01-Team7-24/OurProject.git
%cd OurProject

# Data load

In [3]:
# Set widnow size and k for point adjustment. 0 is f1, 1 is f1_pa

window_size = 20
k_pa = 0

In [ ]:
from Models.lstm_vae.data import get_data_windows

# Get data for training, calibration and testing in form of numpy arrays. Already windowed and normalized.
data_train, data_cal, label_cal, data_test, label_test = get_data_windows(window_size, k_pa)

print(data_train.shape)
print(data_cal.shape, label_cal.shape)
print(data_test.shape, label_test.shape)

# Model load

In [6]:
# Set parameters to load the model
x_dim = data_cal.shape[2]
lstm_h_dim = 10
z_dim = 10

In [ ]:
from Models.lstm_vae.model import LSTM_VAE

model = LSTM_VAE(window_size, x_dim, lstm_h_dim, z_dim, dtype='float32')
model.compile()
model.load_weights('./trained_models/lstm_vae/lstm_vae_ckpt')

# Model calibration

In [ ]:
score_cal = model.anomaly_score(data_cal)
label_cal = label_cal[:score_cal.shape[0]]

In [ ]:
optimize = True
param_grid = {'estimator__C': np.logspace(0, 5, 6), 'estimator__gamma': np.logspace(-5, 0, 6)}

if optimize:
  cal_search = GridSearchCV(CalibratedClassifierCV(SVC(probability=True), cv=3), param_grid, cv=3, verbose=1, scoring='f1')
  cal_search.fit(score_cal, label_cal)
  cal_model = cal_search.best_estimator_
  print(cal_search.best_params_)
else:
  cal_model = CalibratedClassifierCV(SVC(probability=True, C=10, gamma=1e-3))
  cal_model.fit(score_cal, label_cal[:score_cal.shape[0]])

# Model evaluation

In [ ]:
score_test = model.anomaly_score(data_test)
label_test = label_test[:score_test.shape[0]]

y_pred_test = cal_model.predict(score_test)
y_score_test = cal_model.predict_proba(score_test)[:, 1]

In [ ]:
f1 = sklearn.metrics.f1_score(label_test, y_pred_test)
f1

In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(label_test, y_score_test)
roc_auc = sklearn.metrics.auc(fpr, tpr)

plt.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend()
plt.show()

roc_auc

In [ ]:
precison, recall, thresholds = sklearn.metrics.precision_recall_curve(label_test, y_score_test)
prc_auc = sklearn.metrics.auc(recall, precison)

plt.plot(recall, precison, label='PRC curve (area = %0.4f)' % prc_auc)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve')
plt.legend()
plt.show()

prc_auc